# Opening a Ramen Shop in Downtown Toronto

In [1]:
import pandas as pd
import numpy as np

from pandas import Series,DataFrame

import requests
from bs4 import BeautifulSoup

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering librar

Scrape Wikipedia web page for List of Canadian postal codes

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_get = requests.get(url)
html = BeautifulSoup(url_get.text,"html.parser")
table = html.find_all('table', class_="wikitable sortable")
df = pd.read_html(str(table))[0]
df = df[df.Borough != "Not assigned"]

df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [3]:
import geocoder
#g = geocoder.google('Mountain View, CA') #google does not work
g = geocoder.arcgis('Mountain View, CA')
g = g.latlng
print(g)

[37.389670000000024, -122.08159999999998]


In [4]:
import geocoder

def get_latlon(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
for idx, row in df.iterrows():
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(df.loc[idx,'Postal code']))   
    lat_lng_coords = g.latlng
    lat = lat_lng_coords[0]
    lng = lat_lng_coords[1]
    df.loc[idx,'Latitude'] = lat
    df.loc[idx,'Longitude'] = lng      
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.752935,-79.335641
3,M4A,North York,Victoria Village,43.728102,-79.311890
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.723265,-79.451211
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

Since the university of Tronto has the largest student enrollments, we set it as the center of our research.

In [6]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Map all the boroughs in greater Toronto area

In [7]:
# create map of Tronto using latitude and longitude values
map_toronto = folium.Map(location=[lat, lng], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Focus on the downtown of Toronto where most densly populated

In [8]:
toronto_data = df[df['Borough'].str.contains('Toronto',case=False)].reset_index(drop=True)

toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390
2,M5B,Downtown Toronto,Garden District / Ryerson,43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


In [9]:
# create map of Tronto using latitude and longitude values
map_toronto = folium.Map(location=[lat, lng], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Let's find out which neighborhood has a college as students are one of the target customer segment

Let's utilize the Foursquare API to explore the neighborhoods and segment them

In [10]:
CLIENT_ID = 'UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX' # your Foursquare ID
CLIENT_SECRET = 'NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [11]:
search_query = 'University'
LIMIT = 100
radius = 1500
print(search_query + ' .... OK!')

University .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX&client_secret=NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK&ll=43.6534817,-79.3839347&v=20180605&query=University&radius=1500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb25e69c546f3001cb29fbf'},
 'response': {'venues': [{'id': '4c6062b612e5c9286e131f4c',
    'name': 'Dundas University Health Clinic',
    'location': {'address': '104-438 University Avenue',
     'lat': 43.65419587934101,
     'lng': -79.38816564112626,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65419587934101,
       'lng': -79.38816564112626}],
     'distance': 349,
     'postalCode': 'M4P 2K8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['104-438 University Avenue',
      'Toronto ON M4P 2K8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d104941735',
      'name': 'Medical Center',
      'pluralName': 'Medical Centers',
      'shortName': 'Medical',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588747935',
    'hasPerk': False},
   {'i

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4c6062b612e5c9286e131f4c,Dundas University Health Clinic,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1588747935,False,104-438 University Avenue,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,M4P 2K8,CA,Toronto,ON,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",NaN,NaN,NaN
1,4d5d275fff226a3110517e69,438 University Ave,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588747935,False,438 University Ave,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,M5G 2K8,CA,Toronto,ON,Canada,"[438 University Ave (at Dundas St W), Toronto ...",at Dundas St W,NaN,NaN
2,4cd044c29d87224bf129543b,University Centre Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588747935,False,NaN,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,NaN
3,4e6565fb1850e238d7048495,University Avenue Plaza,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1588747935,False,University Avenue,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,NaN,CA,Toronto,ON,Canada,"[University Avenue (btwn Richmond & College), ...",btwn Richmond & College,NaN,NaN
4,4c45c6c6da2176b09543637a,University Centre,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588747935,False,393 University Ave.,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,M7A 2K1,CA,Toronto,ON,Canada,"[393 University Ave. (at Armoury St.), Toronto...",at Armoury St.,NaN,NaN


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_university = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_university['categories'] = dataframe_filtered_university.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_university.columns = [column.split('.')[-1] for column in dataframe_filtered_university.columns]

dataframe_filtered_university

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Dundas University Health Clinic,Medical Center,104-438 University Avenue,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,M4P 2K8,CA,Toronto,ON,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",NaN,NaN,4c6062b612e5c9286e131f4c
1,438 University Ave,Office,438 University Ave,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,M5G 2K8,CA,Toronto,ON,Canada,"[438 University Ave (at Dundas St W), Toronto ...",at Dundas St W,NaN,4d5d275fff226a3110517e69
2,University Centre Gym,Gym / Fitness Center,NaN,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,4cd044c29d87224bf129543b
3,University Avenue Plaza,Plaza,University Avenue,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,NaN,CA,Toronto,ON,Canada,"[University Avenue (btwn Richmond & College), ...",btwn Richmond & College,NaN,4e6565fb1850e238d7048495
4,University Centre,Office,393 University Ave.,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,M7A 2K1,CA,Toronto,ON,Canada,"[393 University Ave. (at Armoury St.), Toronto...",at Armoury St.,NaN,4c45c6c6da2176b09543637a
5,Queen & University,Intersection,Queen St W,43.650833,-79.386858,"[{'label': 'display', 'lat': 43.65083280275225...",377,NaN,CA,Toronto,ON,Canada,"[Queen St W (& University Ave), Toronto ON, Ca...",& University Ave,NaN,4c053af4187ec928411db87b
6,393 University Ave,Office,393 University Ave,43.653752,-79.388150,"[{'label': 'display', 'lat': 43.65375248805427...",340,M5G 1E6,CA,Toronto,ON,Canada,"[393 University Ave, Toronto ON M5G 1E6, Canada]",NaN,NaN,4c655857f67ec928e7485727
7,Lakehead University Book Store,Bookstore,500 University Avenue,43.655428,-79.389002,"[{'label': 'display', 'lat': 43.65542796114875...",462,NaN,CA,Orillia,ON,Canada,"[500 University Avenue, Orillia ON, Canada]",NaN,NaN,4f0c5654e4b087b6f750937a
8,Ryerson University Security Centre,Miscellaneous Shop,111 Bond St.,43.652981,-79.378340,"[{'label': 'display', 'lat': 43.65298107973972...",454,NaN,CA,Toronto,ON,Canada,"[111 Bond St., Toronto ON, Canada]",NaN,NaN,502ebc1ce4b0d68f376457b9
9,University of Toronto,University,27 King's College Cir,43.662493,-79.395220,"[{'label': 'display', 'lat': 43.6624934706167,...",1353,M5S 1A1,CA,Toronto,ON,Canada,"[27 King's College Cir (at King's College Rd),...",at King's College Rd,NaN,4ad4c05ef964a52097f620e3


Let's find the buildings of University of Toronto which is the largest number of students in Canada

In [17]:
uoftoronto_data = dataframe_filtered_university[dataframe_filtered_university['name'].str.contains('University of Toronto',case=False)].reset_index(drop=True)

uoftoronto_data.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,University of Toronto,University,27 King's College Cir,43.662493,-79.395220,"[{'label': 'display', 'lat': 43.6624934706167,...",1353,M5S 1A1,CA,Toronto,ON,Canada,"[27 King's College Cir (at King's College Rd),...",at King's College Rd,NaN,4ad4c05ef964a52097f620e3
1,University Of Toronto Residence Parking Garage,Building,89 Chestnut Street,43.654183,-79.385989,"[{'label': 'display', 'lat': 43.65418264783643...",182,NaN,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN,4be2a144d27a20a11fee8f5b
2,University of Toronto Engineering Society,Student Center,10 King's College Rd. - B740,43.660097,-79.394644,"[{'label': 'display', 'lat': 43.660097244179, ...",1134,M5S 3G4,CA,Toronto,ON,Canada,[10 King's College Rd. - B740 (at Galbraith Rd...,at Galbraith Rd.,NaN,4c850e1b2f1c236ad9e85043
3,University of Toronto Arts Centre,Art Gallery,15 Kings College Circle,43.663605,-79.395159,"[{'label': 'display', 'lat': 43.66360538340076...",1444,NaN,CA,Toronto,ON,Canada,"[15 Kings College Circle, Toronto ON, Canada]",NaN,NaN,4bfd97014cf820a1c0b0ecf4
4,university of Toronto dental,Dentist's Office,NaN,43.656965,-79.384256,"[{'label': 'display', 'lat': 43.656965, 'lng':...",388,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,5633a320498e59191a412f3d


# Identify which areas come up with the keyword Japan

In [18]:
search_query = 'Japan'
radius = 1500
print(search_query + ' .... OK!')

Japan .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX&client_secret=NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK&ll=43.6534817,-79.3839347&v=20180605&query=Japan&radius=1500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb25d68006dce001c1427dc'},
 'response': {'venues': [{'id': '4b705d17f964a52065152de3',
    'name': 'Edo Japan',
    'location': {'address': '260 Yonge St',
     'crossStreet': 'Eaton Centre',
     'lat': 43.655554,
     'lng': -79.380344,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.655554,
       'lng': -79.380344}],
     'distance': 369,
     'postalCode': 'M5B 2L9',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['260 Yonge St (Eaton Centre)',
      'Toronto ON M5B 2L9',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d111941735',
      'name': 'Japanese Restaurant',
      'pluralName': 'Japanese Restaurants',
      'shortName': 'Japanese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588747962',
    'hasPerk': False},
   {'id': '4ee112458b81e335

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4b705d17f964a52065152de3,Edo Japan,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1588747962,False,260 Yonge St,Eaton Centre,43.655554,-79.380344,"[{'label': 'display', 'lat': 43.655554, 'lng':...",369,M5B 2L9,CA,Toronto,ON,Canada,"[260 Yonge St (Eaton Centre), Toronto ON M5B 2...",NaN
1,4ee112458b81e335ed32df69,Consulate-General of Japan in Toronto 在トロント日本国...,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1588747962,False,3300-77 King St W,at TD North Tower,43.648004,-79.381876,"[{'label': 'display', 'lat': 43.64800436073, '...",631,M5K 1A1,CA,Toronto,ON,Canada,"[3300-77 King St W (at TD North Tower), Toront...",NaN
2,5b119a2251950e002c1b7c4b,Japan National Tourist Organization - JNTO,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",v-1588747962,False,481 University Ave.,NaN,43.650894,-79.386563,"[{'label': 'display', 'lat': 43.650894, 'lng':...",357,M5G 2E9,CA,Toronto,ON,Canada,"[481 University Ave., Toronto ON M5G 2E9, Canada]",NaN
3,4b7c2a06f964a52071812fe3,Koya Japan,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1588747962,False,Exchange Tower,NaN,43.649099,-79.383597,"[{'label': 'display', 'lat': 43.6490990795264,...",488,NaN,CA,Toronto,ON,Canada,"[Exchange Tower, Toronto ON, Canada]",NaN
4,4c10fe633ce120a1b8ea081c,Japan camera centre,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",v-1588747962,False,777 Bay Street,NaN,43.661566,-79.382300,"[{'label': 'display', 'lat': 43.661566, 'lng':...",909,NaN,CA,Toronto,ON,Canada,"[777 Bay Street, Toronto ON, Canada]",NaN


In [22]:
dataframe.shape

(32, 18)

In [23]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_jp = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_jp['categories'] = dataframe_filtered_jp.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_jp.columns = [column.split('.')[-1] for column in dataframe_filtered_jp.columns]

dataframe_filtered_jp

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Edo Japan,Japanese Restaurant,260 Yonge St,Eaton Centre,43.655554,-79.380344,"[{'label': 'display', 'lat': 43.655554, 'lng':...",369,M5B 2L9,CA,Toronto,ON,Canada,"[260 Yonge St (Eaton Centre), Toronto ON M5B 2...",NaN,4b705d17f964a52065152de3
1,Consulate-General of Japan in Toronto 在トロント日本国...,Embassy / Consulate,3300-77 King St W,at TD North Tower,43.648004,-79.381876,"[{'label': 'display', 'lat': 43.64800436073, '...",631,M5K 1A1,CA,Toronto,ON,Canada,"[3300-77 King St W (at TD North Tower), Toront...",NaN,4ee112458b81e335ed32df69
2,Japan National Tourist Organization - JNTO,Tourist Information Center,481 University Ave.,NaN,43.650894,-79.386563,"[{'label': 'display', 'lat': 43.650894, 'lng':...",357,M5G 2E9,CA,Toronto,ON,Canada,"[481 University Ave., Toronto ON M5G 2E9, Canada]",NaN,5b119a2251950e002c1b7c4b
3,Koya Japan,Japanese Restaurant,Exchange Tower,NaN,43.649099,-79.383597,"[{'label': 'display', 'lat': 43.6490990795264,...",488,NaN,CA,Toronto,ON,Canada,"[Exchange Tower, Toronto ON, Canada]",NaN,4b7c2a06f964a52071812fe3
4,Japan camera centre,Electronics Store,777 Bay Street,NaN,43.661566,-79.382300,"[{'label': 'display', 'lat': 43.661566, 'lng':...",909,NaN,CA,Toronto,ON,Canada,"[777 Bay Street, Toronto ON, Canada]",NaN,4c10fe633ce120a1b8ea081c
5,Made In Japan,Asian Restaurant,NaN,NaN,43.647132,-79.382596,"[{'label': 'display', 'lat': 43.647132, 'lng':...",715,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4c6aef6b9c76d13ae4594d0f
6,Edo Japan,Japanese Restaurant,199 Bay St,Commerce Court,43.647687,-79.379066,"[{'label': 'display', 'lat': 43.64768665764220...",754,NaN,CA,Toronto,ON,Canada,"[199 Bay St (Commerce Court), Toronto ON, Canada]",NaN,4ceeab5013aea143900b6f9f
7,Japan Communications Ltd,Business Service,401-460 Richmond St W,NaN,43.647678,-79.397668,"[{'label': 'display', 'lat': 43.647678, 'lng':...",1281,M5V 1Y1,CA,Toronto,ON,Canada,"[401-460 Richmond St W, Toronto ON M5V 1Y1, Ca...",NaN,5df286aef8f55f0008346d10
8,Teriyaki Experience,Japanese Restaurant,555 University Ave.,at Elm St. (SickKids Hospital),43.657326,-79.386544,"[{'label': 'display', 'lat': 43.65732640808125...",476,M5G 1X8,CA,Toronto,ON,Canada,[555 University Ave. (at Elm St. (SickKids Hos...,NaN,501425c0e4b0e28d993611ed
9,Japango,Sushi Restaurant,122 Elizabeth St.,at Dundas St. W,43.655268,-79.385165,"[{'label': 'display', 'lat': 43.65526771691681...",222,M5G 1P5,CA,Toronto,ON,Canada,"[122 Elizabeth St. (at Dundas St. W), Toronto ...",NaN,4ae7b27df964a52068ad21e3


In [24]:
#Find Japanese organization and businesses excluding the restaurnt category
dataframe_filtered_jp = dataframe_filtered_jp[~dataframe_filtered_jp['categories'].str.contains('restaurant',case=False)].reset_index(drop=True)

dataframe_filtered_jp

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Consulate-General of Japan in Toronto 在トロント日本国...,Embassy / Consulate,3300-77 King St W,at TD North Tower,43.648004,-79.381876,"[{'label': 'display', 'lat': 43.64800436073, '...",631,M5K 1A1,CA,Toronto,ON,Canada,"[3300-77 King St W (at TD North Tower), Toront...",NaN,4ee112458b81e335ed32df69
1,Japan National Tourist Organization - JNTO,Tourist Information Center,481 University Ave.,NaN,43.650894,-79.386563,"[{'label': 'display', 'lat': 43.650894, 'lng':...",357,M5G 2E9,CA,Toronto,ON,Canada,"[481 University Ave., Toronto ON M5G 2E9, Canada]",NaN,5b119a2251950e002c1b7c4b
2,Japan camera centre,Electronics Store,777 Bay Street,NaN,43.661566,-79.382300,"[{'label': 'display', 'lat': 43.661566, 'lng':...",909,NaN,CA,Toronto,ON,Canada,"[777 Bay Street, Toronto ON, Canada]",NaN,4c10fe633ce120a1b8ea081c
3,Japan Communications Ltd,Business Service,401-460 Richmond St W,NaN,43.647678,-79.397668,"[{'label': 'display', 'lat': 43.647678, 'lng':...",1281,M5V 1Y1,CA,Toronto,ON,Canada,"[401-460 Richmond St W, Toronto ON M5V 1Y1, Ca...",NaN,5df286aef8f55f0008346d10
4,Uncle Tetsu's Japanese Cheesecake,Bakery,NaN,NaN,43.644428,-79.381311,"[{'label': 'display', 'lat': 43.64442750617172...",1029,M5J,CA,Toronto,ON,Canada,"[Toronto ON M5J, Canada]",NaN,57c76c1b498e26057391344c
5,Aitas Japanese language school,School,180 Bloor st west,NaN,43.668922,-79.395110,"[{'label': 'display', 'lat': 43.66892203336142...",1940,NaN,CA,Toronto,ON,Canada,"[180 Bloor st west, Toronto ON, Canada]",NaN,4ada36ecf964a520132021e3


In [25]:
dataframe_filtered_jp.name

0    Consulate-General of Japan in Toronto 在トロント日本国...
1           Japan National Tourist Organization - JNTO
2                                  Japan camera centre
3                             Japan Communications Ltd
4                    Uncle Tetsu's Japanese Cheesecake
5                       Aitas Japanese language school
Name: name, dtype: object

The majority of these venues are just the restaurants. But these shops include the word "Japanese" in their name of the business which means they are promoting the Japan brand. Therefore, we should keep the data for our analysis. 

# Let's find out where existing Ramen shops are located

In [26]:
search_query = 'ramen'
radius = 1500
print(search_query + ' .... OK!')

ramen .... OK!


In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX&client_secret=NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK&ll=43.6534817,-79.3839347&v=20180605&query=ramen&radius=1500&limit=100'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb25eae542890001b48941d'},
 'response': {'venues': [{'id': '504bbf2ce4b0168121235cbe',
    'name': 'Sansotei Ramen 三草亭',
    'location': {'address': '179 Dundas St. W',
     'crossStreet': 'btwn Centre Ave. & Chestnut St.',
     'lat': 43.655157467561246,
     'lng': -79.38650067479335,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.655157467561246,
       'lng': -79.38650067479335}],
     'distance': 278,
     'postalCode': 'M5G 1Z8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['179 Dundas St. W (btwn Centre Ave. & Chestnut St.)',
      'Toronto ON M5G 1Z8',
      'Canada']},
    'categories': [{'id': '55a59bace4b013909087cb24',
      'name': 'Ramen Restaurant',
      'pluralName': 'Ramen Restaurants',
      'shortName': 'Ramen',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/ramen_',
       'suffix': '.png'},
      'primary': True}],
    're

In [29]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,504bbf2ce4b0168121235cbe,Sansotei Ramen 三草亭,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1588747975,False,179 Dundas St. W,btwn Centre Ave. & Chestnut St.,43.655157,-79.386501,"[{'label': 'display', 'lat': 43.65515746756124...",278,M5G 1Z8,CA,Toronto,ON,Canada,[179 Dundas St. W (btwn Centre Ave. & Chestnut...,NaN,NaN
1,5793b232498e3514fcc588ba,Lan Zhou Ramen,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1588747975,False,NaN,NaN,43.655093,-79.386711,"[{'label': 'display', 'lat': 43.65509348676232...",286,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
2,5c9aaa3f66f3cd002c95bd58,Hakata Ikkousha Ramen,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1588747975,False,249 Queen Street West,NaN,43.650299,-79.388753,"[{'label': 'display', 'lat': 43.650299, 'lng':...",525,M5V 1Z4,CA,Toronto,ON,Canada,"[249 Queen Street West, Toronto ON M5V 1Z4, Ca...",NaN,NaN
3,4ae354f7f964a520919321e3,Kenzo Ramen,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1588747975,False,138 Dundas St. W,at Elizabeth St.,43.655540,-79.384866,"[{'label': 'display', 'lat': 43.65554007474909...",241,M5G 1C3,CA,Toronto,ON,Canada,"[138 Dundas St. W (at Elizabeth St.), Toronto ...",NaN,NaN
4,58f3a319419a9e1f9b0f6fcf,Ramen Raijin 雷神,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1588747975,False,24 Wellesley St W,at Yonge St,43.665191,-79.385974,"[{'label': 'display', 'lat': 43.6651907105415,...",1313,M4Y 1W5,CA,Toronto,ON,Canada,"[24 Wellesley St W (at Yonge St), Toronto ON M...",NaN,NaN


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_ramen = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_ramen['categories'] = dataframe_filtered_ramen.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_ramen.columns = [column.split('.')[-1] for column in dataframe_filtered_ramen.columns]

dataframe_filtered_ramen

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sansotei Ramen 三草亭,Ramen Restaurant,179 Dundas St. W,btwn Centre Ave. & Chestnut St.,43.655157,-79.386501,"[{'label': 'display', 'lat': 43.65515746756124...",278,M5G 1Z8,CA,Toronto,ON,Canada,[179 Dundas St. W (btwn Centre Ave. & Chestnut...,NaN,504bbf2ce4b0168121235cbe
1,Lan Zhou Ramen,Ramen Restaurant,NaN,NaN,43.655093,-79.386711,"[{'label': 'display', 'lat': 43.65509348676232...",286,NaN,CA,NaN,NaN,Canada,[Canada],NaN,5793b232498e3514fcc588ba
2,Hakata Ikkousha Ramen,Ramen Restaurant,249 Queen Street West,NaN,43.650299,-79.388753,"[{'label': 'display', 'lat': 43.650299, 'lng':...",525,M5V 1Z4,CA,Toronto,ON,Canada,"[249 Queen Street West, Toronto ON M5V 1Z4, Ca...",NaN,5c9aaa3f66f3cd002c95bd58
3,Kenzo Ramen,Ramen Restaurant,138 Dundas St. W,at Elizabeth St.,43.655540,-79.384866,"[{'label': 'display', 'lat': 43.65554007474909...",241,M5G 1C3,CA,Toronto,ON,Canada,"[138 Dundas St. W (at Elizabeth St.), Toronto ...",NaN,4ae354f7f964a520919321e3
4,Ramen Raijin 雷神,Ramen Restaurant,24 Wellesley St W,at Yonge St,43.665191,-79.385974,"[{'label': 'display', 'lat': 43.6651907105415,...",1313,M4Y 1W5,CA,Toronto,ON,Canada,"[24 Wellesley St W (at Yonge St), Toronto ON M...",NaN,58f3a319419a9e1f9b0f6fcf
5,Kinton Ramen,Ramen Restaurant,51 Baldwin St.,Beverley St.,43.655794,-79.394047,"[{'label': 'display', 'lat': 43.65579361535061...",854,M5T 1L1,CA,Toronto,ON,Canada,"[51 Baldwin St. (Beverley St.), Toronto ON M5T...",NaN,4fad7406be773af34ad09cca
6,Hakata Ikkousha Ramen - Chicken,Ramen Restaurant,257 Queen Street West,NaN,43.650195,-79.389056,"[{'label': 'display', 'lat': 43.650195, 'lng':...",551,M5V 1Z4,CA,Toronto,ON,Canada,"[257 Queen Street West, Toronto ON M5V 1Z4, Ca...",NaN,5e5162c1282afc00082b0cb3
7,Kinton Ramen,Ramen Restaurant,396 Church St,McGill Street,43.660632,-79.378880,"[{'label': 'display', 'lat': 43.66063169500671...",894,M5B 2A2,CA,Toronto,ON,Canada,"[396 Church St (McGill Street), Toronto ON M5B...",NaN,560b1653498e1a2dfa4af722
8,Touhenboku Ramen 唐変木,Ramen Restaurant,261 Queen St W,at Duncan St,43.650240,-79.389189,"[{'label': 'display', 'lat': 43.65024004579307...",556,M5V 1Z9,CA,Toronto,ON,Canada,"[261 Queen St W (at Duncan St), Toronto ON M5V...",NaN,527c18df11d23b4792c0b610
9,Hokkaido Ramen Santouka らーめん山頭火,Ramen Restaurant,91 Dundas St E,at Church St,43.656435,-79.377586,"[{'label': 'display', 'lat': 43.65643520293576...",607,M5B 1E1,CA,Toronto,ON,Canada,"[91 Dundas St E (at Church St), Toronto ON M5B...",Downtown Toronto,509e9ef6e4b0ab175389a6c5


# Find out Chinese businesses and restaurants

In [31]:
search_query = 'Chinese'
radius = 1500
print(search_query + ' .... OK!')

Chinese .... OK!


In [32]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX&client_secret=NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK&ll=43.6534817,-79.3839347&v=20180605&query=Chinese&radius=1500&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb25e0b0f5968001b26e776'},
 'response': {'venues': [{'id': '53973080498e97c3af5edb92',
    'name': 'ILSC-Toronto Chinese Catholic Centre Campus',
    'location': {'address': '202 St Patrick St',
     'lat': 43.65542737655215,
     'lng': -79.39084784196841,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65542737655215,
       'lng': -79.39084784196841}],
     'distance': 597,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['202 St Patrick St', 'Toronto ON', 'Canada']},
    'categories': [{'id': '52e81612bcbc57f1066b7a48',
      'name': 'Language School',
      'pluralName': 'Language Schools',
      'shortName': 'Language School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588747982',
    'hasPerk': False},
   {'id': '4d06c1ccc2e53704428cc567',
    'name':

In [34]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id
0,53973080498e97c3af5edb92,ILSC-Toronto Chinese Catholic Centre Campus,"[{'id': '52e81612bcbc57f1066b7a48', 'name': 'L...",v-1588747982,False,202 St Patrick St,43.655427,-79.390848,"[{'label': 'display', 'lat': 43.65542737655215...",597,CA,Toronto,ON,Canada,"[202 St Patrick St, Toronto ON, Canada]",NaN,NaN,NaN,NaN
1,4d06c1ccc2e53704428cc567,North-East Chinese Restaurant 華北美食,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1588747982,False,476 Dundas St.,43.653185,-79.396677,"[{'label': 'display', 'lat': 43.6531846832903,...",1026,CA,Toronto,ON,Canada,"[476 Dundas St. (Huron), Toronto ON, Canada]",Huron,NaN,NaN,NaN
2,4b622bf1f964a5200b3a2ae3,Chinese Traditional Buns,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1588747982,False,536 Dundas St. W,43.652714,-79.399006,"[{'label': 'display', 'lat': 43.65271411616852...",1216,CA,Toronto,ON,Canada,"[536 Dundas St. W (at Spadina Ave), Toronto ON...",at Spadina Ave,M5T 1H3,Kensington Market,NaN
3,4b072e9df964a52009f922e3,Sky Dragon Chinese Restaurant 龍翔酒樓,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1588747982,False,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",1149,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,NaN,NaN
4,4fd3ea3ee4b0191b9c2b1aff,Rosewood Chinese Cuisine,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1588747982,False,463 Dundas St. W.,43.653171,-79.396710,"[{'label': 'display', 'lat': 43.65317123090316...",1029,CA,Toronto,ON,Canada,"[463 Dundas St. W. (at Larch St.), Toronto ON ...",at Larch St.,M5T 1G8,NaN,NaN


In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_ch = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_ch['categories'] = dataframe_filtered_ch.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_ch.columns = [column.split('.')[-1] for column in dataframe_filtered_ch.columns]

dataframe_filtered_ch

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,ILSC-Toronto Chinese Catholic Centre Campus,Language School,202 St Patrick St,43.655427,-79.390848,"[{'label': 'display', 'lat': 43.65542737655215...",597,CA,Toronto,ON,Canada,"[202 St Patrick St, Toronto ON, Canada]",NaN,NaN,NaN,53973080498e97c3af5edb92
1,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,"[{'label': 'display', 'lat': 43.6531846832903,...",1026,CA,Toronto,ON,Canada,"[476 Dundas St. (Huron), Toronto ON, Canada]",Huron,NaN,NaN,4d06c1ccc2e53704428cc567
2,Chinese Traditional Buns,Chinese Restaurant,536 Dundas St. W,43.652714,-79.399006,"[{'label': 'display', 'lat': 43.65271411616852...",1216,CA,Toronto,ON,Canada,"[536 Dundas St. W (at Spadina Ave), Toronto ON...",at Spadina Ave,M5T 1H3,Kensington Market,4b622bf1f964a5200b3a2ae3
3,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",1149,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,NaN,4b072e9df964a52009f922e3
4,Rosewood Chinese Cuisine,Chinese Restaurant,463 Dundas St. W.,43.653171,-79.396710,"[{'label': 'display', 'lat': 43.65317123090316...",1029,CA,Toronto,ON,Canada,"[463 Dundas St. W. (at Larch St.), Toronto ON ...",at Larch St.,M5T 1G8,NaN,4fd3ea3ee4b0191b9c2b1aff
5,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",282,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",in University Centre,NaN,NaN,4f04779a02d5cce0cfc06151
6,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",300,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",at University Ave,M5G 1C7,NaN,4b2027b5f964a520f82d24e3
7,Winner BBQ Chinese Cuisine DT 贼好吃的烤肉饭,Chinese Restaurant,384 Yonge St Unit 52,43.659320,-79.382794,"[{'label': 'display', 'lat': 43.65932, 'lng': ...",656,CA,Toronto,ON,Canada,"[384 Yonge St Unit 52, Toronto ON M5B 1S8, Can...",NaN,M5B 1S8,NaN,5e02b9409ad0bf000791cd8b
8,Toronto Chinese Academy,University,133 Richmond Street West,43.650140,-79.384857,"[{'label': 'display', 'lat': 43.65014006138457...",379,CA,Toronto,ON,Canada,"[133 Richmond Street West (York Street), Toron...",York Street,M5H 2L3,Financial District,5202d246498e9649c88150b7
9,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,43.655281,-79.385337,"[{'label': 'display', 'lat': 43.65528126342919...",229,CA,Toronto,ON,Canada,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...",Dundas St.,NaN,NaN,52a7ae41498eed3af4d0a3fa


In [36]:
dataframe_filtered_ch.name

0         ILSC-Toronto Chinese Catholic Centre Campus
1                  North-East Chinese Restaurant 華北美食
2                            Chinese Traditional Buns
3                  Sky Dragon Chinese Restaurant 龍翔酒樓
4                            Rosewood Chinese Cuisine
5             Chinese Visa Application Service Center
6                       Hong Shing Chinese Restaurant
7               Winner BBQ Chinese Cuisine DT 贼好吃的烤肉饭
8                             Toronto Chinese Academy
9                        Yueh Tung Chinese Restaurant
10                     Chinese Freemasons Association
11                                     Chinese Bakery
12                              Chinese Gospel Church
13                        Chinese Presbyterian Church
14                                 Tasty Chinese Food
15                                      Chinese Herbs
16                     Toronto Chinese Baptist Church
17                  Chinese Railroad Workers Memorial
18                       Goo

In [37]:
dataframe_filtered_ch

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,ILSC-Toronto Chinese Catholic Centre Campus,Language School,202 St Patrick St,43.655427,-79.390848,"[{'label': 'display', 'lat': 43.65542737655215...",597,CA,Toronto,ON,Canada,"[202 St Patrick St, Toronto ON, Canada]",NaN,NaN,NaN,53973080498e97c3af5edb92
1,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,"[{'label': 'display', 'lat': 43.6531846832903,...",1026,CA,Toronto,ON,Canada,"[476 Dundas St. (Huron), Toronto ON, Canada]",Huron,NaN,NaN,4d06c1ccc2e53704428cc567
2,Chinese Traditional Buns,Chinese Restaurant,536 Dundas St. W,43.652714,-79.399006,"[{'label': 'display', 'lat': 43.65271411616852...",1216,CA,Toronto,ON,Canada,"[536 Dundas St. W (at Spadina Ave), Toronto ON...",at Spadina Ave,M5T 1H3,Kensington Market,4b622bf1f964a5200b3a2ae3
3,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",1149,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,NaN,4b072e9df964a52009f922e3
4,Rosewood Chinese Cuisine,Chinese Restaurant,463 Dundas St. W.,43.653171,-79.396710,"[{'label': 'display', 'lat': 43.65317123090316...",1029,CA,Toronto,ON,Canada,"[463 Dundas St. W. (at Larch St.), Toronto ON ...",at Larch St.,M5T 1G8,NaN,4fd3ea3ee4b0191b9c2b1aff
5,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",282,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",in University Centre,NaN,NaN,4f04779a02d5cce0cfc06151
6,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",300,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",at University Ave,M5G 1C7,NaN,4b2027b5f964a520f82d24e3
7,Winner BBQ Chinese Cuisine DT 贼好吃的烤肉饭,Chinese Restaurant,384 Yonge St Unit 52,43.659320,-79.382794,"[{'label': 'display', 'lat': 43.65932, 'lng': ...",656,CA,Toronto,ON,Canada,"[384 Yonge St Unit 52, Toronto ON M5B 1S8, Can...",NaN,M5B 1S8,NaN,5e02b9409ad0bf000791cd8b
8,Toronto Chinese Academy,University,133 Richmond Street West,43.650140,-79.384857,"[{'label': 'display', 'lat': 43.65014006138457...",379,CA,Toronto,ON,Canada,"[133 Richmond Street West (York Street), Toron...",York Street,M5H 2L3,Financial District,5202d246498e9649c88150b7
9,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,43.655281,-79.385337,"[{'label': 'display', 'lat': 43.65528126342919...",229,CA,Toronto,ON,Canada,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...",Dundas St.,NaN,NaN,52a7ae41498eed3af4d0a3fa


# Map the University of Tronoto locations, Japanese landmarks (organization), direct competitors (Ramen shops ) and indirect competitors (Chinese)

Let's make the university of Toronto as the center of the map.

In [38]:
address = 'University of Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.663461999999996, -79.39775965337452.


In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='University of Tronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese landmarks as green circle markers
for lat, lng, label in zip(dataframe_filtered_jp.lat, dataframe_filtered_jp.lng, dataframe_filtered_jp.categories):
# Show the name of the venue instead of the category in the popup
#for lat, lng, label in zip(dataframe_filtered_jp.lat, dataframe_filtered_jp.lng, dataframe_filtered_jp.name):        
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# add the existing Ramen restaurants as orange circle markers
for lat, lng, label in zip(dataframe_filtered_ramen.lat, dataframe_filtered_ramen.lng, dataframe_filtered_ramen.categories):
# Show the name of the venue instead of the category in the popup
#for lat, lng, label in zip(dataframe_filtered_ch.lat, dataframe_filtered_ch.lng, dataframe_filtered_ch.name):   
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the Chinese restaurants as orange circle markers
for lat, lng, label in zip(dataframe_filtered_ch.lat, dataframe_filtered_ch.lng, dataframe_filtered_ch.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)    
       

# display map
venues_map      

# Let's investigate the existing Ramen restaurants: Kenzo Ramen as an example

Kenzo Ramen appeared to be the only restaurnt which serve Ramen as main dish

In [40]:
venue_id = '4ae354f7f964a520919321e3' # ID of Kenzo Ramen
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ae354f7f964a520919321e3?client_id=UECSPXWJJBZGG1GUPTYOCAL1124V0LYILPX1WYTYLNFRAYKX&client_secret=NN4J1EZZW5IRDSQDPPOJU2DMT5GJU2UWMP0IJG4Q4P1UU2UK&v=20180605'

In [41]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ae354f7f964a520919321e3',
 'name': 'Kenzo Ramen',
 'contact': {'phone': '4162051155', 'formattedPhone': '(416) 205-1155'},
 'location': {'address': '138 Dundas St. W',
  'crossStreet': 'at Elizabeth St.',
  'lat': 43.65554007474909,
  'lng': -79.38486599350118,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.65554007474909,
    'lng': -79.38486599350118}],
  'postalCode': 'M5G 1C3',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['138 Dundas St. W (at Elizabeth St.)',
   'Toronto ON M5G 1C3',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/kenzo-ramen/4ae354f7f964a520919321e3',
 'categories': [{'id': '55a59bace4b013909087cb24',
   'name': 'Ramen Restaurant',
   'pluralName': 'Ramen Restaurants',
   'shortName': 'Ramen',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/ramen_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 62},
 'url': 'http://www.kenz

Check the overall rating of Kenzo Ramen

In [42]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.8


The rating is not so good, which is a good news.

In [43]:
print(result['response']['venue']['url'])

http://www.kenzoramen.ca


In [44]:
print(result['response']['venue']['reasons'])

{'count': 1, 'items': [{'summary': 'Lots of people like this place', 'type': 'general', 'reasonName': 'rawLikesReason'}]}


In [45]:
print(result['response']['venue']['canonicalUrl'])

https://foursquare.com/v/kenzo-ramen/4ae354f7f964a520919321e3


In [46]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb25eca542890001b48ca3d'},
 'response': {'tips': {'count': 62,
   'items': [{'id': '560824ab498e8712c89289e3',
     'createdAt': 1443374251,
     'text': "Tasty ramen in many different varieties. I've tried the tonkotsu and the tantanmen and they were both quite delicious and authentic. Also, they have TAKOYAKI BALLS!!! Osaka style :)",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/560824ab498e8712c89289e3',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '4749894',
      'firstName': 'Celio',
      'lastName': 'B',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/5ORIRRIMLO53FVF0.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [47]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [48]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\akihi\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Tasty ramen in many different varieties. I've tried the tonkotsu and the tantanmen and they were both quite delicious and authentic. Also, they have TAKOYAKI BALLS!!! Osaka style :)",0,0,560824ab498e8712c89289e3,Celio,B,NaN,4749894


There is only one user review. The user review is positive. Let's see if this user review is credible.

In [49]:
# define tips URL
user_id = "4749894"

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"Tasty ramen in many different varieties. I've tried the tonkotsu and the tantanmen and they were both quite delicious and authentic. Also, they have TAKOYAKI BALLS!!! Osaka style :)",0,0,560824ab498e8712c89289e3
1,The quesillos are best in the morning.,1,0,557115de498e0eedc25a5dec
2,"Good service, quiet and quaint.",1,0,557115bb498e8182a6b8195e
3,"Gorgeous, quiet, spacious, lovely trails.",1,0,55711552498eac6e55d5b897
4,The line moves surprisingly fast.,1,0,4e2c07281f6ebe54425967a8
5,"Contemporary art center for the prefecture of Osaka. weekly or bi-monthly exhibitions by top tier local, regional, national and international artists. Pretty cool space. right above passport center.",1,0,4dbdc39bced7ea5898d972d1
6,"My favorite is the #6, ginger pork set. Delicious!!!",1,0,4dbdc2d6ced7ea5898d96f62
7,"May 3rd to 8th: Chisato Hori solo exhibition of watercolor and Nihonga paintings. Tue-Sat 12PM to 7PM, Sun 12PM to 5PM sharp.",2,0,4dbdc1ffced7ea5898d96c50
8,Different art exhibits every week by Japanese and foreign artists.,2,0,4d0f43cb7177b1f79a664a22
9,"The spicy sauce is a real treat. The owners are friendly and warm. This place is nice and comfortable. Oh yeah, the burritos are great!",3,0,4cf63517e308236adebae4ac


In [50]:
total_agreed = tips_filtered['agreeCount'].sum()
total_disagreed = tips_filtered['disagreeCount'].sum()
print('The review has total agree counts:'+str(total_agreed))
print('The review has total disagree counts:'+str(total_disagreed))

The review has total agree counts:13
The review has total disagree counts:0


The review is credible based on how others agreed or disagreed on his reviews.

# Market situation

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District / Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosed

In [53]:
print(toronto_venues.shape)
toronto_venues.head()

(1599, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,Regent Park / Harbourfront,43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,Regent Park / Harbourfront,43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,Regent Park / Harbourfront,43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,Regent Park / Harbourfront,43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [54]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,67,67,67,67,67,67
Brockton / Parkdale Village / Exhibition Place,44,44,44,44,44,44
Business reply mail Processing Centre,100,100,100,100,100,100
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,66,66,66,66,66,66
Central Bay Street,51,51,51,51,51,51
Christie,12,12,12,12,12,12
Church and Wellesley,84,84,84,84,84,84
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [55]:
# one hot encoding
#The get_dummies() function is used to convert categorical variable into dummy/indicator variables.
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(toronto_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))
# use loc to reorder
toronto_onehot = toronto_onehot.loc[:, cols]

toronto_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park / Harbourfront,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Find out all the restaurant types

In [56]:
toronto_restauranttypes = [col for col in toronto_onehot.columns if 'Restaurant' in col]
toronto_restauranttypes

['American Restaurant',
 'Asian Restaurant',
 'Belgian Restaurant',
 'Brazilian Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Cuban Restaurant',
 'Eastern European Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Hawaiian Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Modern European Restaurant',
 'Molecular Gastronomy Restaurant',
 'Moroccan Restaurant',
 'New American Restaurant',
 'North Indian Restaurant',
 'Peruvian Restaurant',
 'Portuguese Restaurant',
 'Ramen Restaurant',
 'Restaurant',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Theme Restaurant',
 'Tibetan Restaura

In [57]:
toronto_onehot_church = toronto_onehot.groupby(['Neighborhood']).sum().sort_values(by=['Japanese Restaurant'], ascending=False)
toronto_onehot_church

,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Church and Wellesley,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Business reply mail Processing Centre,0,2,0,1,0,0,3,0,0,0,...,0,0,0,0,2,0,0,1,0,0
Toronto Dominion Centre / Design Exchange,0,3,0,1,0,0,2,0,0,0,...,0,0,1,0,1,0,0,1,0,0
Stn A PO Boxes,0,2,0,1,0,0,3,0,0,0,...,0,0,0,0,2,0,0,1,0,0
Commerce Court / Victoria Hotel,0,4,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,1
First Canadian Place / Underground city,0,4,0,1,0,0,3,0,0,0,...,0,0,1,0,1,0,0,1,0,0
University of Toronto / Harbord,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
Garden District / Ryerson,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
toronto_onehot_restjp = toronto_onehot[['Neighborhood','Japanese Restaurant','Ramen Restaurant','Sushi Restaurant']].groupby(['Neighborhood']).sum().sort_values(by=['Japanese Restaurant'], ascending=False)
toronto_onehot_restjp

,Japanese Restaurant,Ramen Restaurant,Sushi Restaurant
Neighborhood,,,
Church and Wellesley,6,1,4
Business reply mail Processing Centre,4,1,2
Toronto Dominion Centre / Design Exchange,4,0,1
Stn A PO Boxes,4,1,2
Commerce Court / Victoria Hotel,4,0,0
First Canadian Place / Underground city,4,0,1
University of Toronto / Harbord,2,0,0
Garden District / Ryerson,2,2,0
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,2,1,1


In [59]:
toronto_onehot_restas = toronto_onehot[['Neighborhood','Chinese Restaurant','Asian Restaurant','Taiwanese Restaurant','Vietnamese Restaurant']].groupby(['Neighborhood']).sum().sort_values(by=['Chinese Restaurant'], ascending=False)
toronto_onehot_restas

,Chinese Restaurant,Asian Restaurant,Taiwanese Restaurant,Vietnamese Restaurant
Neighborhood,,,,
St. James Town / Cabbagetown,2,0,1,0
University of Toronto / Harbord,1,0,0,0
Toronto Dominion Centre / Design Exchange,1,2,0,0
Garden District / Ryerson,1,0,0,1
Runnymede / Swansea,0,0,0,0
Parkdale / Roncesvalles,0,0,0,1
Queen's Park / Ontario Provincial Government,0,0,0,0
Regent Park / Harbourfront,0,0,0,0
Richmond / Adelaide / King,0,1,0,0


In [60]:
toronto_onehot['Japanese Restaurant'].sum()

39

In [61]:
toronto_onehot['Ramen Restaurant'].sum()

7

In [62]:
toronto_onehot['Sushi Restaurant'].sum()

25

In [63]:
toronto_onehot['Chinese Restaurant'].sum()

5

In [64]:
toronto_onehot['Taiwanese Restaurant'].sum()

1

In [65]:
toronto_onehot['Asian Restaurant'].sum()

17

In [66]:
toronto_onehot['Korean Restaurant'].sum()

1

In [67]:
toronto_onehot['Vietnamese Restaurant'].sum()

7

In [68]:
toronto_onehot['Thai Restaurant'].sum()

23

In [69]:
toronto_onehot['American Restaurant'].sum()

25

In [70]:
toronto_onehot['Italian Restaurant'].sum()

37

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,...,0.000000,0.00,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.00,0.014925
1,Brockton / Parkdale Village / Exhibition Place,0.022727,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Business reply mail Processing Centre,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
3,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.019608,0.019608,0.000000,0.00,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.011905,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.011905
7,Commerce Court / Victoria Hotel,0.000000,0.040000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.038462,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [72]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Coffee Shop     0.09
1  Cocktail Bar    0.04
2  Hotel           0.03
3  Breakfast Spot  0.03
4  Beer Bar        0.03


----Brockton / Parkdale Village / Exhibition Place----
                    venue  freq
0  Coffee Shop             0.09
1  Café                    0.07
2  Pizza Place             0.05
3  Thrift / Vintage Store  0.05
4  Gift Shop               0.05


----Business reply mail Processing Centre----
                 venue  freq
0  Coffee Shop          0.08
1  Hotel                0.05
2  Café                 0.04
3  Japanese Restaurant  0.04
4  Restaurant           0.03


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
               venue  freq
0  Café               0.06
1  Coffee Shop        0.06
2  Restaurant         0.05
3  Park               0.05
4  French Restaurant  0.05


----Central Bay Street----
                       venue  freq
0  Coffee Shop  

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Lounge,Bakery,Breakfast Spot,Restaurant,Cheese Shop,Hotel
1,Brockton / Parkdale Village / Exhibition Place,Coffee Shop,Café,Thrift / Vintage Store,Diner,Pizza Place,Gift Shop,Sandwich Place,Boutique,Italian Restaurant,Brewery
2,Business reply mail Processing Centre,Coffee Shop,Hotel,Japanese Restaurant,Café,Restaurant,Asian Restaurant,Italian Restaurant,Theater,Steakhouse,Bookstore
3,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,Coffee Shop,Café,Restaurant,French Restaurant,Park,Bar,Speakeasy,Lounge,Italian Restaurant,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Café,Middle Eastern Restaurant,Plaza,Clothing Store,Restaurant,Bubble Tea Shop,Sandwich Place,Hotel,Mexican Restaurant
5,Christie,Grocery Store,Café,Park,Baby Store,Athletics & Sports,Coffee Shop,Candy Store,Playground,Fish & Chips Shop,Fish Market
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Dance Studio,Grocery Store,Pub,Gay Bar,Hotel,Men's Store
7,Commerce Court / Victoria Hotel,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,American Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Deli / Bodega
8,Davisville,Dessert Shop,Pizza Place,Café,Sandwich Place,Thai Restaurant,Coffee Shop,Italian Restaurant,Farmers Market,Sushi Restaurant,Toy / Game Store
9,Davisville North,Breakfast Spot,Department Store,Gym,Park,Food & Drink Shop,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop


Let's examine three neighborhoods which meet the criterium

In [75]:
neighborhoods_venues_sorted.take([4, 6, 13])

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Coffee Shop,Café,Middle Eastern Restaurant,Plaza,Clothing Store,Restaurant,Bubble Tea Shop,Sandwich Place,Hotel,Mexican Restaurant
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Dance Studio,Grocery Store,Pub,Gay Bar,Hotel,Men's Store
13,Garden District / Ryerson,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Café,Cosmetics Shop,Restaurant,Hotel,Bar,Italian Restaurant


Clustering Neighborhoods

In [76]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [77]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041,0,Pub,Café,Athletics & Sports,Coffee Shop,Performing Arts Venue,Theater,Seafood Restaurant,Mexican Restaurant,Food Truck,French Restaurant
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390,0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Discount Store,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar,Italian Restaurant
2,M5B,Downtown Toronto,Garden District / Ryerson,43.657491,-79.377529,0,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Café,Cosmetics Shop,Restaurant,Hotel,Bar,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Coffee Shop,Café,American Restaurant,Seafood Restaurant,Cosmetics Shop,Gastropub,Cocktail Bar,Theater,Italian Restaurant,Hotel
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Trail,Pub,Yoga Studio,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [78]:
toronto_merged.isna().any()

Postal code               False
Borough                   False
Neighborhood              False
Latitude                  False
Longitude                 False
Cluster Labels            False
1st Most Common Venue     False
2nd Most Common Venue     False
3rd Most Common Venue     False
4th Most Common Venue     False
5th Most Common Venue     False
6th Most Common Venue     False
7th Most Common Venue     False
8th Most Common Venue     False
9th Most Common Venue     False
10th Most Common Venue    False
dtype: bool

In [79]:
#Delete the row with NaN values
#toronto_merged = toronto_merged.drop([19], axis=0)

In [80]:
toronto_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041,0,Pub,Café,Athletics & Sports,Coffee Shop,Performing Arts Venue,Theater,Seafood Restaurant,Mexican Restaurant,Food Truck,French Restaurant
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390,0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Discount Store,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar,Italian Restaurant
2,M5B,Downtown Toronto,Garden District / Ryerson,43.657491,-79.377529,0,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Café,Cosmetics Shop,Restaurant,Hotel,Bar,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Coffee Shop,Café,American Restaurant,Seafood Restaurant,Cosmetics Shop,Gastropub,Cocktail Bar,Theater,Italian Restaurant,Hotel
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Trail,Pub,Yoga Studio,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,0,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Lounge,Bakery,Breakfast Spot,Restaurant,Cheese Shop,Hotel
6,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,0,Coffee Shop,Café,Middle Eastern Restaurant,Plaza,Clothing Store,Restaurant,Bubble Tea Shop,Sandwich Place,Hotel,Mexican Restaurant
7,M6G,Downtown Toronto,Christie,43.668602,-79.420387,0,Grocery Store,Café,Park,Baby Store,Athletics & Sports,Coffee Shop,Candy Store,Playground,Fish & Chips Shop,Fish Market
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650542,-79.384116,0,Coffee Shop,Café,Restaurant,Clothing Store,Salad Place,Gym,Sushi Restaurant,Deli / Bodega,Thai Restaurant,Hotel
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.664910,-79.438664,0,Park,Smoke Shop,Pharmacy,Brazilian Restaurant,Café,Liquor Store,Bank,Bakery,Furniture / Home Store,Pool


In [81]:
match = ['Japanese Restaurant','Ramen Restaurant','Sushi Restaurant' ]

def string_finder(row, words):
    if any(word in field for field in row for word in words):
        return True
    return False

toronto_rest = toronto_merged.astype(str).apply(string_finder, words=match)
toronto_rest

Postal code               False
Borough                   False
Neighborhood              False
Latitude                  False
Longitude                 False
Cluster Labels            False
1st Most Common Venue     False
2nd Most Common Venue     True 
3rd Most Common Venue     True 
4th Most Common Venue     False
5th Most Common Venue     True 
6th Most Common Venue     False
7th Most Common Venue     True 
8th Most Common Venue     True 
9th Most Common Venue     True 
10th Most Common Venue    True 
dtype: bool

Find out which neighborhoods have Japanese restaurants as the top 10 common venue

In [82]:
neighbor_jp_rest = toronto_merged[toronto_merged.isin(['Japanese Restaurant', 'Sushi Restaurant','Ramen Restaurant']).any(axis=1)]
neighbor_jp_rest

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390,0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Discount Store,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar,Italian Restaurant
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650542,-79.384116,0,Coffee Shop,Café,Restaurant,Clothing Store,Salad Place,Gym,Sushi Restaurant,Deli / Bodega,Thai Restaurant,Hotel
13,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.646923,-79.381626,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Deli / Bodega,Asian Restaurant
16,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,43.648283,-79.378750,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,American Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Deli / Bodega
25,M6R,West Toronto,Parkdale / Roncesvalles,43.647840,-79.448388,0,Coffee Shop,Food & Drink Shop,Pizza Place,Restaurant,Sushi Restaurant,Eastern European Restaurant,Thai Restaurant,Bakery,Café,Convenience Store
26,M4S,Central Toronto,Davisville,43.703327,-79.385649,0,Dessert Shop,Pizza Place,Café,Sandwich Place,Thai Restaurant,Coffee Shop,Italian Restaurant,Farmers Market,Sushi Restaurant,Toy / Game Store
27,M5S,Downtown Toronto,University of Toronto / Harbord,43.663281,-79.398088,0,Café,Italian Restaurant,Restaurant,Park,Japanese Restaurant,Bookstore,Bubble Tea Shop,French Restaurant,Flower Shop,Chinese Restaurant
28,M6S,West Toronto,Runnymede / Swansea,43.649725,-79.482692,0,Café,Pizza Place,Coffee Shop,Gastropub,Italian Restaurant,Dessert Shop,Pub,Falafel Restaurant,Sushi Restaurant,Restaurant
32,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,43.640539,-79.397435,0,Coffee Shop,Café,Restaurant,French Restaurant,Park,Bar,Speakeasy,Lounge,Italian Restaurant,Japanese Restaurant
34,M5W,Downtown Toronto,Stn A PO Boxes,43.648700,-79.385450,0,Coffee Shop,Hotel,Japanese Restaurant,Café,Restaurant,Asian Restaurant,Italian Restaurant,Theater,Steakhouse,Bookstore


Find out if Ramen Restaurants are in the top 10 common venue in any of the neighborhood

In [83]:
neighbor_jp_ramen = toronto_merged[toronto_merged.isin(['Ramen Restaurant']).any(axis=1)]
neighbor_jp_ramen

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Ramen restaurants are not in the top 10 venue in any neighborhoods

First, Let's map all the clusters

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1], # this cause an error in Python3
        color=rainbow[int(cluster)-1],
        fill=True,
        #fill_color=rainbow[cluster-1], # this cause an error in Python3
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters